In [0]:
from tqdm import tqdm
from model.nn import *
from pyspark.sql import functions as F
from components.config import *
from components.helper import *
import pickle
import os
import shutil
%load_ext autoreload
%autoreload 2

In [0]:
property_name = "Borgata"

In [0]:
path = "data_science_mart.tierimminent_cleaned."
table_name = f"{property_name}"+"_trip_cleaned_return"
cleaned_df = spark.table(f"{path}{table_name}")

unsucess_id = cleaned_df.groupby('train_guest_id').agg(F.sum('label').alias('s')).where('s = 0').sample(withReplacement=False, fraction=0.7, seed=269)
train_final = cleaned_df.join(unsucess_id, on = 'train_guest_id', how='leftanti').select(*cleaned_df.columns)
train_final = train_final.where('(change_assigned != transactiondate) or change_assigned is null')
train_final = train_final.distinct()
train_final.count()

In [0]:
train_tensor = []
label_tensor = []
length_tensor = []
id_tensor = []

sanity_train_length = []

features_scaled = []
for i in range(len(FEATURE_NAMES_RETURN)):
    features_scaled.append(FEATURE_NAMES_RETURN[i]+'_scaled')

for i in tqdm(range(365, 0, -1)):
    cus_list = train_final.filter(train_final.remaining_days == i).select('train_guest_id').distinct()
    temp_df = train_final.join(cus_list, (train_final.train_guest_id == cus_list.train_guest_id) & (train_final.remaining_days >= i), how = 'inner').drop(train_final.train_guest_id)
    temp_df = temp_df.withColumn('rn', F.row_number().over(Window.partitionBy('train_guest_id').orderBy('remaining_days'))).where('rn <= 6')
    train_np, label, length, train_id = create_sequences_and_labels(temp_df, features_scaled, 6)

    print(cus_list.count(), len(train_np))
    
    sanity_train_length.append(len(train_np))
    train_tensor.extend(train_np)
    label_tensor.extend(label)
    length_tensor.extend(length)
    id_tensor.extend(train_id)

In [0]:
sanity_train_length = np.array(sanity_train_length)
train_tensor = np.array(train_tensor)
label_tensor = np.array(label_tensor)
length_tensor = np.array(length_tensor)
id_tensor = np.array(id_tensor)

In [0]:
np_dict = {
    "sanity_train_length": sanity_train_length,
    "train_tensor": train_tensor,
    "label_tensor": label_tensor,
    "length_tensor": length_tensor,
    "id_tensor": id_tensor
}

local_tmp_dir = "/Workspace/Users/609399@mgmresorts.com/pickle_files"
dbfs_target_dir = "dbfs:/mnt/proddatalake/dev/TierImminent/intermediate_data/borgata_return_01312025"
os.makedirs(local_tmp_dir, exist_ok=True)

with open(f"{local_tmp_dir}/np_dict.pkl", "wb") as f:
    pickle.dump(np_dict, f)

dbutils.fs.cp(f"file:{local_tmp_dir}/np_dict.pkl", f"{dbfs_target_dir}_np_dict.pkl")

shutil.rmtree(local_tmp_dir)


# New customer


In [0]:
path = "data_science_mart.tierimminent_cleaned."
table_name = f"{property_name}"+"_trip_cleaned_new"
cleaned_df = spark.table(f"{path}{table_name}")

unsucess_id = cleaned_df.groupby('train_guest_id').agg(F.sum('label').alias('s')).where('s = 0').sample(withReplacement=False, fraction=0.5, seed=269)
train_final = cleaned_df.join(unsucess_id, on = 'train_guest_id', how='leftanti').select(*cleaned_df.columns)
train_final = train_final.where('(change_assigned != transactiondate) or change_assigned is null')
train_final = train_final.distinct()
train_final = train_final.cache()
train_final.count()

In [0]:
train_final.where('label = 1').count()

In [0]:
train_final.describe().display()

In [0]:
train_tensor = []
label_tensor = []
length_tensor = []
id_tensor = []

sanity_train_length = []

features_scaled = []
for i in range(len(FEATURE_NAMES_NEW)):
    features_scaled.append(FEATURE_NAMES_NEW[i]+'_scaled')

for i in tqdm(range(365, 0, -1)):
    cus_list = train_final.filter(train_final.remaining_days == i).select('train_guest_id').distinct()
    temp_df = train_final.join(cus_list, (train_final.train_guest_id == cus_list.train_guest_id) & (train_final.remaining_days >= i), how = 'inner').drop(train_final.train_guest_id)
    temp_df = temp_df.withColumn('rn', F.row_number().over(Window.partitionBy('train_guest_id').orderBy('remaining_days'))).where('rn <= 6')
    train_np, label, length, train_id = create_sequences_and_labels(temp_df, features_scaled, 6)

    print(cus_list.count(), len(train_np))
    
    sanity_train_length.append(len(train_np))
    train_tensor.extend(train_np)
    label_tensor.extend(label)
    length_tensor.extend(length)
    id_tensor.extend(train_id)

In [0]:
sanity_train_length = np.array(sanity_train_length)
train_tensor = np.array(train_tensor)
label_tensor = np.array(label_tensor)
length_tensor = np.array(length_tensor)
id_tensor = np.array(id_tensor)

In [0]:
np_dict = {
    "sanity_train_length": sanity_train_length,
    "train_tensor": train_tensor,
    "label_tensor": label_tensor,
    "length_tensor": length_tensor,
    "id_tensor": id_tensor
}

local_tmp_dir = "/Workspace/Users/609399@mgmresorts.com/pickle_files"
dbfs_target_dir = "dbfs:/mnt/proddatalake/dev/TierImminent/intermediate_data/borgata_new_02102025"
os.makedirs(local_tmp_dir, exist_ok=True)

with open(f"{local_tmp_dir}/np_dict.pkl", "wb") as f:
    pickle.dump(np_dict, f)

dbutils.fs.cp(f"file:{local_tmp_dir}/np_dict.pkl", f"{dbfs_target_dir}_np_dict.pkl")

shutil.rmtree(local_tmp_dir)
